### Homework 2 - Data from the Web

#### 1. Extract data

Obtain the 200 top-ranking universities in www.topuniversities.com (ranking 2018). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the details page. Store the resulting dataset in a pandas DataFrame.

In [ ]:
%matplotlib inline  
import requests
import pandas as pd
import json
import difflib
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

SOURCE="www.topuniversities.com"
LIST=SOURCE + "/university-rankings/world-university-rankings/2018"
DATA_SOURCE="www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt"

LIMIT=200

In [ ]:
# Some helpers used to clean data from scrapping
def clean_numerical_data(data):
    return int(data.replace('\n', '').replace(',', ''))

def clean_percentage_data(data):
    return int(data.replace('%', '')) / 100

def safe_parse(page, selector):
    try:
        return clean_numerical_data(page.select(selector)[0].text)
    except:
        return None

In [ ]:
# Here we use CSS Selector to easily extract data from web pages
def extract_faculty_members(page):
    total = safe_parse(page, "div.total.faculty div.number")
    international = safe_parse(page, "div.inter.faculty div.number")
    return total, international
    
def extract_student_members(page):
    total = safe_parse(page, "div.total.student div.number")
    international = safe_parse(page, "div.total.inter  div.number")
    return total, international

def extract_data(university):
    url = SOURCE + university['url']
    data = requests.get("http://" + url).text
    page = BeautifulSoup(data, "html5lib")
    
    total_faculty, inter_faculty = extract_faculty_members(page)
    total_student, inter_student = extract_student_members(page)

    university['total_faculty'] = total_faculty
    university['inter_faculty'] = inter_faculty
    university['total_student'] = total_student
    university['inter_student'] = inter_student
    
    return university

In [ ]:
# Extract JSON from url, retrieve only data
def extract_list(url):
    data = requests.get("http://" + url).text
    return json.loads(data)['data']
    
university_list = extract_list(DATA_SOURCE)[:LIMIT]

In [ ]:
university_list = [extract_data(university) for university in university_list]

In [ ]:
KEEP_TOP = ['title', 'rank_display', 'cc', 'region', 'total_faculty', 'total_student', 'inter_faculty', 'inter_student']
university_list = [ {key: university[key] for key in KEEP_TOP} for university in university_list]

In [ ]:
df_top = pd.DataFrame(university_list)
df_top.index = df_top['title']
df_top

#### 1.b Which are the best universities in term of:

##### (a) ratio between faculty members and students

There are two possible ratio that can be used to answer this question, we can either use the **total** statistics or the **international** ones, we though it is more meaningful to use the **total** to answers this question 

In [ ]:
def staff_student_ratio(df):
    df['ratio_faculty_student'] = df['total_student'] /  df['total_faculty']
    return df.sort_values('ratio_faculty_student')

staff_student_ratio(df_top)['ratio_faculty_student'].head().plot(kind='bar', figsize=(7,7), title="Ratio of student per staff member")

We see that universities with the best ratio between faculty staff and students are located in the US or in GB but that this metric is not closely linked with the rank of the university

##### (b) ratio of international students?

In [ ]:
def inter_staff_ratio(df):
    df['ratio_inter_staff'] = df['inter_faculty'] / df['total_faculty']
    
def inter_student_ratio(df):
    df['ratio_inter_student'] = df['inter_student'] / df['total_student']
    return df.sort_values('ratio_inter_student', ascending=False)

inter_student_ratio(df_top)['ratio_inter_student'].head().plot(kind='bar', figsize = (7,7), title="Ratio of international student")
inter_staff_ratio(df_top)

Here, we can see that the most international university are mostly located in Europe.

We now need to compute the same ratio, but this time, we'll add some aggregation

##### Country aggregation

The first aggregation we want to perform is aggretion by country, we simply `groupby` the dataframe using the **cc** column and then call the same methods previously created (`staff_student_ratio` and `inter_student_ratio`)

In [ ]:
df_country = df_top.groupby('cc')['inter_faculty', 'inter_student', 'total_faculty', 'total_student'].sum()

In [ ]:
staff_student_ratio(df_country)['ratio_faculty_student'].head().plot(kind='bar', figsize=(7, 7), title="Proportion of student per staff member by Country")

In [ ]:
inter_student_ratio(df_country)['ratio_inter_student'].head().plot(kind='bar', figsize=(7, 7), title="Ratio of international student per country")

##### Region aggregation

The second aggregation we want to perform is an aggregation by region, same as before, we simply `groupby` and call the methods on the newly created dataframe.

In [ ]:
df_region = df_top.groupby('region')['inter_faculty', 'inter_student', 'total_faculty', 'total_student'].sum()

In [ ]:
staff_student_ratio(df_region)['ratio_faculty_student'].head().plot(kind='bar',  figsize=(7, 7), title="Proportion of student per faculty member by region")

In [ ]:
inter_student_ratio(df_region)['ratio_inter_student'].head().plot(kind='bar', figsize=(7,7), title="Ratio of international student per region")

### Data from timeshighereducation

Obtain the 200 top-ranking universities in www.timeshighereducation.com (ranking 2018). Repeat the analysis of the previous point and discuss briefly what you observed.

In [ ]:
SOURCE="www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"
university_list_2 = extract_list(SOURCE)[:LIMIT]

In [ ]:
df_times = pd.DataFrame(university_list_2)

df_times['total_student'] = df_times['stats_number_students'].apply(clean_numerical_data)
df_times['ratio_faculty_student'] = df_times['stats_student_staff_ratio'].apply(float)
df_times['ratio_inter_student'] = df_times['stats_pc_intl_students'].apply(clean_percentage_data)
df_times['title'] = df_times['name']
df_times['rank_display'] = df_times['rank']
df_times['cc'] = df_times['location']

KEEP_TIMES = ['title', 'rank_display', 'cc', 'total_student', 'ratio_faculty_student', 'ratio_inter_student', 'scores_international_outlook_rank']
df_times = df_times[KEEP_TIMES]
df_times.index = df_times['title']

Here we graph the same data we plotted for the topuniversities ranking, except that we don't have the region for each university, one way we could fix that is to find the region for a given country code.

In [ ]:
df_times.sort_values('ratio_inter_student', ascending=False)['ratio_inter_student'].head().plot(kind='bar', figsize = (7,7), title="Ratio of international student")

For this metric, we get almost the same data as from topuniversities, except for Luxembourg, because it has a rank greater than 200 according to TopUniversities

In [ ]:
df_times.sort_values('ratio_faculty_student')['ratio_faculty_student'].head().plot(kind='bar', figsize = (7,7), title="Ratio of student per faculty member")

Here we have some different result, some university have a much better ratio according to timeshigherrank than with topuniversities.
We now are going to compare those metric aggregated by country:

In [ ]:
df_times_country = df_times.groupby('cc')['ratio_faculty_student', 'ratio_inter_student'].median()

In [ ]:
df_times_country.sort_values('ratio_faculty_student')['ratio_faculty_student'].head().plot(kind='bar',  figsize=(7, 7), title="Proportion of student per faculty member by region")

The university name matching is the hardest part of this homework we decided to make it as concervative as possible (that is to say, we decided to have quality matches over the number of matches), we proceeded in a few steps:
 - The first step is to remove common name in all title (University, Technology, ...) but simply removing them would lead to mismatch (for instance if California Institute of Technologie and University of California, only California would remain and match the two together, even if there are not the same). To avoid this issue, we simply replace each of this term by a integer (that would not make many difference when we compute the distance between two words)
 - Then, we mapped this stripped name from one dataframe to the stripped names of the other dataframe, the first thing is to look for exact matches (and there are many of them), then we use the `difflib.get_close_matches` function to retrieve the closest match. Unfortunately we had lot's of mismatches and this is why we decided to do a reverse matching.
 - The reverse match works as follow: once we found the closest match (let's call it RESULT) from dataframe ONE to dataframe TWO, we find the closest match of RESULT into dataframe ONE, if this is equals to the original name, then it's a match.
 
 Using this technique, all the matches we have are correct, but we only have 152 out of 200 universities, we can explain this by many ways:
  - The university is above 200 in one ranking and below in the other (many cases)
  - University name are written using acronymes in one ranking and in with the full name in the other
  - German names (Munchen, Munich)


In [ ]:
import re
# Here we remove some very commons words that reduce the "uniqueness" of the name
def remove_common_word(word):
    return re.sub(r'\(.*\)', '', word).replace('University', '0').replace('Technology', '1').replace('Institute', '2').replace(' of ', ' ').replace('Scuola', '').replace(' the ', ' ').rstrip().lstrip().lower()

# There are many ways to perform the matching, but sometimes, it's simply not possible
# This is the case for instance for the University of Paris-Sud, which is not in the < 200 ranking according to topuniversities
# Or luxembourg (which is not even in the list)
def match_university_name(name, currents, others):
    tmp = [x for x in others if  name == x]
    if len(tmp): return tmp[0]
    
    matches = difflib.get_close_matches(name, others)
    result = 'undefined'
    if len(matches) != 0:
        tmp = matches[0]
        match_reverse = difflib.get_close_matches(tmp, currents)
        if len(match_reverse):
            if match_reverse[0] == name:
                result = tmp

    return result

df_top['cleared_title'] =  df_top['title'].apply(lambda x: remove_common_word(x))
df_times['cleared_title'] = df_times['title'].apply(lambda x: remove_common_word(x))

df_times['title_mapped'] = df_times['cleared_title'].map(lambda x: match_university_name(x, df_times['cleared_title'],df_top['cleared_title']))
df_top['title_mapped'] =  df_top['cleared_title'] 

df_times.index = df_times['title_mapped']
df_top.index = df_top['title_mapped']

In [ ]:
df_join = df_times.join(df_top, on='title_mapped', how='inner', lsuffix='_times', rsuffix='_top')
df_join[ 'scores_international_outlook_rank'] = df_join[ 'scores_international_outlook_rank'].apply(pd.to_numeric)

**when a university is strong in its international dimension, can you observe a consistency both for students and faculty members**

In [ ]:
df_corr= df_join[['ratio_inter_student_times', 'ratio_inter_staff', 'scores_international_outlook_rank']]
df_corr.corr()

Here we calculated the correlation between the ratio of international students versus the international university rank, and the ratio of international staff member versus the international university rank. The correlations for both ratio are close to -70%, when the international ratio increases, the rank decreases. It seems to be consistent because more international students/staffs a university has, the better its international rank is. This alsoshows us that the international ranking of an university is more correlated to the number of students than to the number of staffs in the university. Still we cannot be sure of that because our dataset is limited to 200 entries and both result are close to -70%.

In [ ]:
df_corr = df_join[['ratio_inter_student_times', 'ratio_inter_staff', 'rank_display_times']].copy()
df_corr['rank'] = df_corr['rank_display_times'].map(lambda x: int(x.replace('=', '')))
df_corr.corr()

Here we do not take the international ranking into account, but  the global one. This time the rank is not enough correlated to the ratio of international students or staffs in the university to be able to conclude anything.

#### Can you find the best university taking in consideration both rankings? Explain your approach.

We decided to simply take the average of both global ranks because we thought it was more relevant as they are probable already computed from the other features of the university.

In [ ]:
df_rank = df_join[['rank_display_top', 'rank_display_times']].copy()
df_rank.index = df_join['title_times']
df_rank['rank_display_top'] = df_rank['rank_display_top'].apply(lambda x: int(x.replace('=', '')))
df_rank['rank_display_times'] = df_rank['rank_display_times'].apply(lambda x: int(x.replace('=', '')))
df_rank['avg'] = df_rank.mean(axis=1)

df_rank.sort_values('avg').head()

Hence, according to this average, the best university would be Stanford University, EPFL would be number 12 